# User-clickable Forced Photometry from the stack

This notebook retrieves a coadd image and a single science exposure from Stripe 82 data, and sets up Firefly to return forced photometry from the LSST stack by clicking on a location in an image.

### Imports

Imports for Python 2/3 compatibility.

In [ ]:
from __future__ import print_function, division, absolute_import

Standard library imports, and allowing the notebook server to find needed modules.

In [ ]:
import sys
import os
import concurrent.futures
import tempfile

Imports from the LSST stack.

In [ ]:
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage
import lsst.log
from lsst.daf.persistence import Butler

Imports from Astropy.

In [ ]:
from astropy.table import Table, vstack, join, Column
from astropy.time import Time

Import the Firefly Python client.

In [ ]:
sys.path.insert(0,'/home/shupe/projects/firefly_client/firefly_client')
import firefly_client
from firefly_client import FireflyClient

Import functions for forced photometry.

In [ ]:
from dax_utils import get_image_table
from stripe82phot import make_refcat, parse_phot_table, do_phot

### Firefly setup

Set up the FireflyClient to point to the local Firefly server.

In [ ]:
mychannel = 'stripe82'
fc = FireflyClient('lsst-sui-tomcat01.ncsa.illinois.edu:8080', channel=mychannel)

At this point, make sure VPN to vpn.ncsa.illinois.edu is active. Then, open a browser window to http://lsst-sui-tomcat01.ncsa.illinois.edu:8080/firefly/lsst-pdac-triview.html;wsch=stripe82

Helper function to do forced photometry for a dataId and to return None if an exception is encountered. The dataId for SDSS data is specified by run, field, camcol, filter.

In [ ]:
def do_one(dataId):
    global src_cat
    try:
        rval = do_phot(dataId=dataId, refCat=src_cat)
    except:
        return
    return(rval)

Define a function to retrieve forced photometry for a specified RA, Dec, filter_name.

In [ ]:
def fetch_forcedphot(ra, dec, filter_name):
    lsst.log.setLevel('', lsst.log.INFO)
    import numpy as np
    np.seterr(all='ignore')
    lsst.log.info('Querying for list of science exposures at this point')
    df_f = get_image_table(ra, dec, filter_name, 'Science_Ccd_Exposure')
    if df_f is None:
        lsst.log.error('Query returned null')
        return
    df_f['img_url'] = df_f.scienceCcdExposureId.map(lambda x: 
            'http://lsst-qserv-dax01.ncsa.illinois.edu:5000/image/v0/calexp/id?id=' + str(x))
    lsst.log.info('Retrieved table of {} science exposures'.format(len(df_f)))
    ids = [{'run':row.run, 'field':row.field, 'camcol':row.camcol, 
            'filter':row.filterName.encode()} 
            for index, row in df_f.iterrows()]
    global src_cat
    # Make a catalog for the specified RA, Dec
    src_cat = make_refcat([ra], [dec])
    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
        results = executor.map(do_one, ids)
    # Discard None results where an exception was raised in the forced photometry task
    afwtabs = [r for r in results if r is not None]
    # Convert afwTable objects to Astropy tables, applying calibration
    tbl_list = [parse_phot_table(t) for t in afwtabs]
    alltabs = vstack(tbl_list)
    # merge the
    intab = Table.from_pandas(df_f)
    outtab = join(alltabs, intab, keys=['run','camcol','field','filterName'],
                  join_type='left')
    t = Time(outtab['expMidpt'], format='isot', scale='utc')
    outtab['mjd'] = t.mjd
    outtab.sort('mjd')
    outtab['ra'] = outtab['coord_ra'].to('deg')
    outtab['dec'] = outtab['coord_dec'].to('deg')
    mycols = ['mjd','base_PsfFlux_flux','base_PsfFlux_fluxSigma', 'ra','dec',
            'expMidpt','run','field','camcol','filterName',
            'objectId','base_RaDecCentroid_x', 'base_RaDecCentroid_y', 
            'psfMag', 'psfMagErr', 'img_url']
    outtab.keep_columns(mycols)
    newtab = outtab[mycols]
    return(newtab)

The callback function applies `fetch_forcedphot` when activated in Firefly.

In [ ]:
def callback_forcedphot(event):
    global src_cat
    if 'wpt' in event['data']:
        wpt = event['data']['wpt']
        wdata = wpt.split(';')
        ra = float(wdata[0])
        dec = float(wdata[1])
        outtab = fetch_forcedphot(ra, dec, myfilter)
        if outtab is None:
            lsst.log.error('No photometry returned')
            return
        outtab.write('fout.tbl', format='ipac')
        with open('fout.tbl', 'r') as original: 
            data = original.read()
        with open('fout.tbl', 'w') as modified: 
            modified.write("\datasource = img_url\n" + 
                           "\positionCoordColumns = ra;dec;EQ_J2000\n" +
                           data)
        tval = fc.upload_file('fout.tbl')
        fc.show_table(tval, tbl_id='Forced Phot')
        fc.show_xyplot(tbl_id='Forced Phot', xCol='mjd', yCol='base_PsfFlux_flux',
                      yError='base_PsfFlux_fluxSigma', yOptions='grid')

Add the callback (on the Firefly server).

In [ ]:
plistner = fc.add_listener(callback_forcedphot)

### Coordinates and filter specification, and image retrieval

A very nice variable source is at RA=45.804433, Dec=0.905573. (For name resolvers, it is V\* GI Cet.) Here it is used just for image retrieval, along with the specified filter name.

Other high-amplitude variables with P > 50 days, rAmpl > 3.0:

* 25.612807 0.291621, ID=1340590, rAmpl=3.217, P=2958d. (2SLAQ J014227.07+001729.8 -- Star.)
* 28.930942 0.468687, ID=1261335, rAmpl=4.146, P=3321 d. (V\* FL Cet -- CV of AM Her type (polar) -- the period is really 87 minutes.)

Shorter periods:

* 10.62013 -0.869339, ID=196130, iAmpl=8.059, P=0.6d. (SDSS J004228.83-005210.4, White Dwarf.)

QSOs:

* 2.87667 0.964417 ID=68411, iAmpl=2.218, P=2875 days, ([VV2006] J001130.4+005751 -- Quasar.)
* 319.572495 0.221337, id=2655567, iAmpl=3.113, P=1453 days

* UGC 2479 is 45.167525 0.020461

You may specify any coordinates in the DC_2013 Stripe 82 processing, and any filter from u,g,r,i,z.

In [ ]:
myra = 25.6
mydec = 0.3
myfilter = u'z'

Read coadds from the butler -- locate with a DAX call

In [ ]:
df_deep = get_image_table(myra, mydec, myfilter, 'DeepCoadd')
len(df_deep)

In [ ]:
df_deep

In [ ]:
dbutler = Butler('/datasets/sdss/preprocessed/dr7/sdss_stripe82_00/coadd')

In [ ]:
coadd = dbutler.get('deepCoadd', tract=df_deep.tract.values[0],
                       patch=df_deep.patch.values[0],
                        filter=df_deep.filterName.values[0])

### Display images and interact with them

Add the extension to make a button labeled 'Forced Phot' when in Point (Lock by click) mode in Firefly.

Define a display using the LSST stack, pointing to the same location.

In [ ]:
display1 = afwDisplay.Display(frame=1, backend='firefly', name=mychannel)

In [ ]:
display1.mtv(coadd, title='Deep {}, Id {}'.format(myfilter, df_deep.deepCoaddId.values[0]))

In [ ]:
fc.add_extension('POINT', '1', 'Forced Phot', 'fetch forced photometry',
                extension_id='fphot');

* Go to Firefly tab, click inside deep image.
* Select source for forced photometry. 
* Click on 'Forced Phot' option above image.

Forced photometry table should appear in Firefly after 15-20 seconds.